In [1]:
from textmining_utility.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np
import sys, os
import importlib
import textmining.cluster_analysis as cluster_analysis
import textmining.lexicons as lexicons
import textmining.machine_learning as machine_learning
import textmining.significance_testing as significance_testing
import iesta.loader as loader
import iesta.properties as prop  
import iesta.processor as proc  
import iesta.feature_extractor as fe
from iesta.machine_learning.dataloader import IESTAData, METHODOLOGY

from collections import Counter

importlib.reload(cluster_analysis)
importlib.reload(lexicons)
importlib.reload(machine_learning)
importlib.reload(significance_testing)
importlib.reload(loader)
importlib.reload(prop)
importlib.reload(proc)
importlib.reload(fe)


<module 'iesta.feature_extractor' from 'c:\\users\\elba_ro\\documents\\projects\\conf22-style-transfer\\iesta\\feature_extractor.py'>

In [2]:
liberal_each_dataloader = IESTAData(ideology=prop.LIBERAL_IDEOLOGY, methodology=METHODOLOGY.EACH)

liberal_each_dataloader.load()



textmining-utility INFO     File already created. Loading file...
textmining-utility INFO     File already created. Loading file...


In [3]:
liberal_each_dataloader.pivot_df

effect,effective,ineffective,okay,provocative
split,,,,
test,248,5349,969,92
training,1722,37713,6835,522
validation,613,10749,1828,204


In [4]:
liberal_each_dataloader.pivot_binary_effect

binary_effect,effective,ineffective
split,,
test,248,6410
training,1722,45070
validation,613,12781


In [5]:
liberal_training_each_df  = liberal_each_dataloader.data_df[liberal_each_dataloader.data_df["split"] == "training"].copy()
len(liberal_training_each_df)

46792

In [6]:
liberal_training_each_df.columns.tolist()

['id',
 'debate_id',
 'p_name',
 'effects',
 'effect_count',
 'top_effect',
 'effect',
 'category',
 'round',
 'argument',
 'split',
 'binary_effect']

In [20]:
import dataclasses
class TransformerBasedFeaturePipeline(Pipeline):
    
    def __init__(self,
                 input: "[Collection]" = None,
                 load_default_pipe_configs = True,
                 extended_pipe_configs:dict = None,
                 save_output= False,
                 out_path = None,
                 argument_col:str = "argument"
                 ):
        super().__init__(input,
                 load_default_pipe_configs,
                 extended_pipe_configs,
                 save_output,
                 out_path)
        self.argument_col= argument_col
        
    def process_input(self) -> list:
        processed = []
        txt_df = self.input[["id", self.argument_col]].copy()
        txt_df =txt_df.rename(columns={"id": "input_id", self.argument_col: "text"},)
        
        for idx, row in txt_df.iterrows():
            processed.append((row.text, {"input_id": row.input_id}))

        return processed

    def init_and_run(self):
        
        self.add_annotation_pipe(name = "senter",                  save_output= False,is_spacy=True, is_native=True)
        #self.add_annotation_pipe(name = "EmotionPipeOrchestrator", save_output= True, is_spacy=True)
        self.add_annotation_pipe(name = "HedgePipeOrchestrator",   save_output= True, is_spacy=True)
        #self.add_annotation_pipe(name = "ToxicityOrchestrator",    save_output= True, is_spacy=True)

        # annotate the input
        #self.set_spacy_language_model("en_core_web_md")
        self.annotate()
        # save annotations when "save_output" is set to True
        self.save()

In [17]:
import pandas  as pd
test_df = pd.DataFrame([{"id": 1, "argument": "This is a very hard time, I am devastated!"},
                         {"id": 2, "argument": "You can not get your tiny brain to work on this so stupid!!"},
                         {"id": 3, "argument": "The amonium Nitrate was sitting there for ages."},
                         {"id": 4, "argument": "I love you and I love how you look"}
                       ])

In [18]:
pipeline = TransformerBasedFeaturePipeline(save_output= True,
                 out_path = "../data/extracted_features/test.parquet")
pipeline.set_input(test_df)
#pipeline.set_input(test_df)
pipeline.init_and_run()
pipeline.out_df.head()

textmining-utility INFO     adding pipe with name EmotionPipeOrchestrator
textmining-utility INFO     adding pipe with name EmotionPipeOrchestrator
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.emotion
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.emotion
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     adding pipe with code emotion_hartmann_component
textmining-utility INFO     adding pipe with code emotion_hartmann_component
textmining-utility INFO     Defining pipe default and spacy stacks
textmining-utility INFO     Defining pipe default and spacy stacks
  0%|                                                                                            | 0/2 [00:00<?, ?it/s]textmining-utility INFO     saving annotations of Pipe(name='EmotionPipeOrchestrator', save_output=True, is_spacy=True, is_nati

,input_id,emotion_hartmann_anger,emotion_hartmann_anger_count,emotion_hartmann_anger_ratio,emotion_hartmann_disgust,emotion_hartmann_disgust_count,emotion_hartmann_disgust_ratio,emotion_hartmann_fear,emotion_hartmann_fear_count,emotion_hartmann_fear_ratio,...,emotion_hartmann_neutral,emotion_hartmann_neutral_count,emotion_hartmann_neutral_ratio,emotion_hartmann_sadness,emotion_hartmann_sadness_count,emotion_hartmann_sadness_ratio,emotion_hartmann_surprise,emotion_hartmann_surprise_count,emotion_hartmann_surprise_ratio,emotion_hartmann_label
0,1,0.001895,0.0,0.0,0.001441,0.0,0.0,0.003758,0.0,0.0,...,0.004330,0.0,0.0,0.973397,1.0,1.0,0.012989,0.0,0.0,sadness
1,2,0.723535,1.0,1.0,0.222484,0.0,0.0,0.007573,0.0,0.0,...,0.015128,0.0,0.0,0.015681,0.0,0.0,0.013002,0.0,0.0,anger
2,3,0.021493,0.0,0.0,0.283587,0.0,0.0,0.004941,0.0,0.0,...,0.656143,1.0,1.0,0.018515,0.0,0.0,0.011650,0.0,0.0,neutral
3,4,0.002313,0.0,0.0,0.002388,0.0,0.0,0.000476,0.0,0.0,...,0.004757,0.0,0.0,0.011659,0.0,0.0,0.003272,0.0,0.0,joy


In [ ]:
pipeline = TransformerBasedFeaturePipeline(save_output= True,
                 out_path = "../data/extracted_features/hedge_liberal_training_each.parquet")
pipeline.set_input(liberal_training_each_df)
#pipeline.set_input(test_df)
pipeline.init_and_run()
pipeline.out_df.head()

textmining-utility INFO     adding pipe with name HedgePipeOrchestrator
textmining-utility INFO     adding pipe with name HedgePipeOrchestrator
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.hedge
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.hedge
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     adding pipe with code hedge_component
textmining-utility INFO     adding pipe with code hedge_component
textmining-utility INFO     Defining pipe default and spacy stacks
textmining-utility INFO     Defining pipe default and spacy stacks


In [ ]:
pipeline.spacy_language_model

In [ ]:
spacy.load(pipeline.spacy_language_model, disable=["ner"])

In [ ]:
pipeline_cons.processed_input#annotated_artifacts

In [ ]:
pipeline_cons.out_df

In [ ]:
lib_transformers_df = pd.read_parquet("../data/extracted_features/liberal_transformers_features.parquet")

lib_transformers_df.head()
lib_transformers_df['hedge_dominant'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['emotion_hartmann_label'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['toxicity_dominant'].value_counts().to_frame().plot(kind="bar")

lib_transformers_df[["emotion_hartmann_anger",
                     "emotion_hartmann_disgust",
                    "emotion_hartmann_fear", 
                    "emotion_hartmann_joy",
                    "emotion_hartmann_neutral",
                    "emotion_hartmann_surprise",
                    "emotion_hartmann_sadness"]].plot.hist(bins=12, alpha=0.5)